In [1]:
import ctf,time,random
import numpy as np
import numpy.linalg as la
from ctf import random as crandom
glob_comm = ctf.comm()

In [9]:
class UnitTests:
    
    def test_updateOmega(self):
        I = random.randint(3,5)  #random dimensions
        J = random.randint(3,5)
        K = random.randint(3,5)
        sparsity = .2
        assert(updateOmega(I,J,K,sparsity).shape == (I,J,K))
        print("passed test: test_updateOmega")
        pass
        
    def test_3d_purturb1(self):
        
        I = random.randint(3,5)  #random dimensions
        J = random.randint(3,5)
        K = random.randint(3,5)
        r = 2     
        regParam = 0
        sparsity = .2
        
        ctf.random.seed(10)
        # generate factor matrices
        U = ctf.random.random((I,r))
        V= ctf.random.random((J,r))
        W= ctf.random.random((K,r))
        
        omega = updateOmega(I,J,K,sparsity)
        
        T = ctf.tensor((I,J,K))
        T.i("ijk") << U.i("iu")*V.i("ju")*W.i("ku")
        
        # purturb the first factor matrix
        U += crandom.random((I,r))*.01
        # call updateU function
        nU = updateU(T,U,V,W,regParam,omega,I,J,K,r)
        
        nT = ctf.tensor((I,J,K))
        nT.i("ijk") << nU.i("iu")*V.i("ju")*W.i("ku")
        
        print("nT: ",nT)
        print("T: ",T)
    
        assert(ctf.all(ctf.abs(nT - T < 1e-10)))
        #print("New: ", nT)
        #print("Old: ", T)
        print("passed test: test_3d_purturb1")

        
    def runAllTests(self):
        self.test_updateOmega()
        self.test_3d_purturb1()

In [3]:
def normalize(Z,r):
    norms = ctf.tensor(r)
    norms.i("u") << Z.i("pu")*Z.i("pu")
    norms = 1./norms**.5
    X = ctf.tensor(copy=Z)
    Z.set_zero()
    Z.i("pu") << X.i("pu")*norms.i("u")
    print("norms: ",norms)
    return 1./norms

In [4]:
def updateOmega(I,J,K,sparsity):
    '''
    Gets a random subset of rows for each U,V,W iteration
    '''
    Actf = ctf.tensor((I,J,K),sp=True)
    Actf.fill_sp_random(0,1,sparsity)
    omegactf = ((Actf > 0)*ctf.astensor(1.))
    return omegactf


def getIndexFromOmega(omega,I,J,K):
    '''
    Gets a random i,j,k contained in Ω[0...i-1,0...j-1,0...k-1]
    '''
    i = random.randint(0,I-1)
    j = random.randint(0,J-1)
    k = random.randint(0,K-1)
    while omega[i][j][k] != 1:
        i = random.randint(0,I-1)
        j = random.randint(0,J-1)
        k = random.randint(0,K-1)
        
    return (i,j,k)

In [5]:
def updateU(T,U,V,W,regParam,omega,I,J,K,r):
    '''Update U matrix by using the formula'''
    
    M1 = ctf.tensor((J,K,r))
    M1.i("jku") << V.i("ju")*W.i("ku")
    [U_,S_,V_] = ctf.svd(M1.reshape((J*K,r)))
    #S_ = 1./S_
    print("S_before: ",S_)
    S_ = S_/(S_*S_ + regParam*ctf.ones(r))
    print("S_after: ",S_)
    U.set_zero()
    U.i("iu") << V_.i("vu")*S_.i("v")*U_.reshape((J,K,r)).i("jkv")*T.i("ijk")
    #U *= normalize(U,r)
    
    return U
    
    
def updateV(T,U,V,W,regParam,omega,I,J,K,r):
    '''Update V matrix by using the formula'''
    
    M2 = ctf.tensor((I,K,r))
    M2.i("iku") << U.i("iu")*W.i("ku")
    [U_,S_,V_] = ctf.svd(M2.reshape((I*K,r)))
    #S_ = 1./S_
    S_ = S_/(S_*S_ + regParam*ctf.ones(r))
    V.set_zero()
    V.i("ju") << V_.i("vu")*S_.i("v")*U_.reshape((I,K,r)).i("ikv")*T.i("ijk")
    #normalize(V,r)
    #V *= normalize(V,r)
    
    return V  

def updateW(T,U,V,W,regParam,omega,I,J,K,r):
    '''Update V matrix by using the formula'''
    
    M3 = ctf.tensor((I,J,r))
    M3.i("iju") << U.i("iu")*V.i("ju")
    [U_,S_,V_] = ctf.svd(M3.reshape((I*J,r)))
    #S_ = 1./S_
    S_ = S_/(S_*S_ + regParam*ctf.ones(r))
    W.set_zero()
    W.i("ku") << V_.i("vu")*S_.i("v")*U_.reshape((I,J,r)).i("ijv")*T.i("ijk")
    #W *= normalize(W,r)
    
    return W

In [6]:
def getALSCtf(T,U,V,W,regParam,omega,I,J,K,r):
    '''
    Same thing as above, but CTF
    '''
    it = 0
    E = ctf.tensor((I,J,K))
    E.i("ijk") << T.i("ijk") - U.i("iu")*V.i("ju")*W.i("ku")
    #E.i("ijk") << T.i("ijk") - omega.i("ijk")*U.i("iu")*V.i("ju")*W.i("ku")
    curr_err_norm = ctf.vecnorm(E) + (ctf.vecnorm(U) + ctf.vecnorm(V) + ctf.vecnorm(W))*regParam
    
    while True:
        U = updateU(T,U,V,W,regParam,omega,I,J,K,r)
        print("U: ",U)
        V = updateV(T,U,V,W,regParam,omega,I,J,K,r) 
        W = updateW(T,U,V,W,regParam,omega,I,J,K,r)
        E.set_zero()
        E.i("ijk") << T.i("ijk") - U.i("iu")*V.i("ju")*W.i("ku")
        #E.i("ijk") << T.i("ijk") - omega.i("ijk")*U.i("iu")*V.i("ju")*W.i("ku")
        next_err_norm = ctf.vecnorm(E) + (ctf.vecnorm(U) + ctf.vecnorm(V) + ctf.vecnorm(W))*regParam
        print("err norm: ",next_err_norm)
        
        if abs(curr_err_norm - next_err_norm) < .001 or it > 10:
            break
            
        print(curr_err_norm, next_err_norm)
        print(next_err_norm/curr_err_norm)
        curr_err_norm = next_err_norm
        it += 1
        
    print("T: ", T)
    ALS = ctf.tensor((I,J,K))
    ALS.i("ijk") <<  U.i("iu")*V.i("ju")*W.i("ku")
    print("ALS: ", ALS)
        
    return U,V,W

In [7]:
def main():
    
    #ut = UnitTests()
    #ut.runAllTests()

    I = random.randint(3,5)
    J = random.randint(3,5)
    K = random.randint(3,5)
    r = 2 
    sparsity = .2
    regParam = 0.001
        
    ctf.random.seed(42)
    U = ctf.random.random((I,r))
    V= ctf.random.random((J,r))
    W= ctf.random.random((K,r))
    
    #U *= normalize(U,r)
    #V *= normalize(V,r)
    #W *= normalize(W,r)
    
    # 3rd-order tensor
    #T = ctf.tensor((I,J,K),sp=True)
    #T.fill_sp_random(0,1,1)
    T = ctf.tensor((I,J,K))
    #T.fill_random(0,1)
    T.i("ijk") << U.i("iu")*V.i("ju")*W.i("ku")
    U = ctf.random.random((I,r))
    V= ctf.random.random((J,r))
    W= ctf.random.random((K,r))
    print("nU: ",U)
    
    omega = updateOmega(I,J,K,sparsity)
    
    t = time.time()
    
    getALSCtf(T,U,V,W,regParam,omega,I,J,K,r)
    
    print("ALS costs time = ",np.round_(time.time()- t,4))    

In [8]:
main()

NameError: name 'UnitTests' is not defined